# 𝐇𝐲𝐩𝐨𝐭𝐡𝐞𝐭𝐢𝐜𝐚𝐥 𝐃𝐨𝐜𝐮𝐦𝐞𝐧𝐭 𝐄𝐦𝐛𝐞𝐝𝐝𝐢𝐧𝐠 (𝐇𝐲𝐃𝐄)

In a standard retriever-augmented generation (RAG) workflow, retrieval typically involves comparing the embedding vector of a query with the embedding vectors of document chunks to find the most relevant matches. However, this approach has limitations, especially in zero-shot dense retrieval. The challenge arises because query and document embeddings must exist in the same vector space, even though questions and documents are fundamentally different. Queries are often short, ambiguous, or poorly phrased, while documents contain structured, detailed information.



𝐇𝐨𝐰 𝐇𝐲𝐃𝐄 𝐈𝐦𝐩𝐫𝐨𝐯𝐞𝐬 𝐑𝐞𝐭𝐫𝐢𝐞𝐯𝐚𝐥

HyDE addresses this by mapping queries into the document space using hypothetical documents. Instead of directly embedding the question, we use a language model (LLM) to generate a synthetic passage based on the query. We then perform vector search using this generated passage rather than the raw question, leading to better alignment with document embeddings.



𝐖𝐡𝐞𝐧 𝐭𝐨 𝐔𝐬𝐞 𝐇𝐲𝐃𝐄

✅ Boosting retrieval performance when recall is low

✅ Handling large document spaces where standard retrieval struggles

✅ Cross-domain retrieval, especially when documents come from a different domain than the retriever was trained on



If your RAG pipeline has recall issues or operates in a challenging domain, HyDE is definitely worth exploring! 🚀

In [1]:
import openai,os
from openai import AzureOpenAI
from dotenv import load_dotenv
from typing import DefaultDict
from azure.identity import ClientSecretCredential
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

load_dotenv()

/Users/Arun.Mohan/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
def get_access_token():
    """ 
    gets api key by authenticating with azure ad
    """
    credential = ClientSecretCredential(os.getenv('OPENAI_AD_TENANT_ID'), os.getenv('OPENAI_AD_CLIENT_ID'),
                                            os.getenv('OPENAI_AD_CLIENT_SECRET'))
    access_token = credential.get_token(os.getenv('OPENAI_AD_TOKEN_BASE'))
    return access_token.token 

Before getting to hyDe lets embedd some documents in to vector database

In [5]:
Docs = [

"EV batteries primarily use lithium-ion chemistry, known for high energy density and rechargeability. Other components include cobalt, nickel, manganese, and graphite, each influencing performance, cost, and safety.",
"Solid-state batteries are emerging as a promising alternative to conventional lithium-ion batteries. They offer increased energy density, faster charging, and improved safety due to the absence of liquid electrolytes.",
"Legacy automakers like Ford, GM, and Volkswagen have invested billions in EV development, with GM committing to an all-electric future by 2035. These investments cover vehicle platforms, battery plants, and EV-specific R&D.",
"Tech companies like Apple and Xiaomi are also exploring EVs, either through direct vehicle development or software and autonomous systems. Their entry reflects how EVs are becoming a convergence point for the auto and tech industries.",
"Public charging infrastructure is critical for EV adoption, with fast chargers (DCFC) reducing charging times to under 30 minutes. Countries like China and the U.S. are rapidly expanding nationwide networks.",
"Home charging accounts for over 80% of EV charging, making Level 2 residential chargers an essential part of the ecosystem. Incentives often cover installation costs to promote at-home charging convenience.",
"Many EVs are integrated with advanced driver-assistance systems (ADAS), leveraging sensors, AI, and over-the-air updates. Tesla's Autopilot and Rivian’s Driver+ are examples of smart EV technologies evolving toward autonomy.",
"Global EV sales surpassed 14 million units in 2023, representing nearly 18% of all new car sales. Asia, especially China, leads the market, but Europe and North America are also seeing rapid growth",
"EVs produce zero tailpipe emissions, making them a cleaner alternative to internal combustion engine vehicles. However, lifecycle emissions depend on how electricity is generated.",
"Battery recycling and reuse are becoming crucial to EV sustainability. Companies like Redwood Materials are working on systems to recover lithium, cobalt, and nickel from end-of-life batteries to reduce mining and environmental impact."

]

### initialize a embedding model
from langchain_openai import AzureOpenAIEmbeddings

def get_emebeddding_model():

    

    embedding_model = AzureOpenAIEmbeddings(
                # azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME"),
                model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME"),
                azure_endpoint=os.getenv("AZURE_OPENAI_EMBEDDING_BASE"),
                openai_api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
                openai_api_key=get_access_token()
    )
    return embedding_model


embedding_model = get_emebeddding_model()



# Create a vector store with a documents
from langchain_core.vectorstores import InMemoryVectorStore

vectorstore = InMemoryVectorStore.from_texts(
    Docs,
    embedding=embedding_model,
)

First step of HyDE is to ask llm to generate an answer.

In [ ]:
from langchain_core.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI


def get_llm_gen_answer_prompt_template():

    system_message = """"Given a question, generate a paragraph under 30 words of text that answers the question"""

    user_message = """Question: {question}
                      Paragraph:"""

    system_prompt = SystemMessagePromptTemplate.from_template(system_message)
    user_prompt = HumanMessagePromptTemplate.from_template(user_message)

    prompt = ChatPromptTemplate.from_messages(
            [
                system_prompt,
                user_prompt
            ]
        )
    return prompt



def get_llm():

    api_key = get_access_token()

    llm = AzureChatOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_API_BASE"),
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
        api_key=api_key
    )
    return llm



In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain
# Define the LLM
from langchain_openai import ChatOpenAI


## initialize llm
llm = get_llm()

##prompt template
prompt_template = get_llm_gen_answer_prompt_template()


# Create a chain
gen_context_chain = (
        prompt_template
        | llm
        | StrOutputParser()
    )



In [12]:
query = "How is EV sale performing globally?"



# Pass the user input
suggested_context = gen_context_chain.invoke({
        "question": query
    })
print(suggested_context)

Global EV sales are surging, driven by increasing environmental awareness, government incentives, and advancements in battery technology, resulting in record growth and expanding market adoption worldwide.


Now using this response we will search again vector database

In [13]:

def get_chunks(query,k):
    retrieved_documents = vectorstore.similarity_search_with_score(k=k,query=query,sorted=True)

    return retrieved_documents

chunks = get_chunks(query=suggested_context,k=3)
chunks

[(Document(id='7fcad0ff-e779-4382-98d4-c3f4c959a2ba', metadata={}, page_content='Global EV sales surpassed 14 million units in 2023, representing nearly 18% of all new car sales. Asia, especially China, leads the market, but Europe and North America are also seeing rapid growth'),
  0.7600522387425275),
 (Document(id='bafe51a9-7be0-4d6d-b0d1-3b4d6b669faa', metadata={}, page_content='Legacy automakers like Ford, GM, and Volkswagen have invested billions in EV development, with GM committing to an all-electric future by 2035. These investments cover vehicle platforms, battery plants, and EV-specific R&D.'),
  0.5491659680277841),
 (Document(id='4e47c171-c5e1-4626-9459-94d095d4a3fa', metadata={}, page_content="Many EVs are integrated with advanced driver-assistance systems (ADAS), leveraging sensors, AI, and over-the-air updates. Tesla's Autopilot and Rivian’s Driver+ are examples of smart EV technologies evolving toward autonomy."),
  0.5123516244543715)]

In [14]:



def get_retriever_prompt_template():

    system_message = "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know."

    user_message = """Context: {context}
                        Question: {question}
                        Answer:"""

    system_prompt = SystemMessagePromptTemplate.from_template(system_message)
    user_prompt = HumanMessagePromptTemplate.from_template(user_message)

    prompt = ChatPromptTemplate.from_messages(
            [
                system_prompt,
                user_prompt
            ]
        )
    return prompt


In [15]:

## initialize llm
llm = get_llm()

##prompt template
prompt_template = get_retriever_prompt_template()

# Create a chain
answer_chain = (
        prompt_template
        | llm
        | StrOutputParser()
    )


In [16]:
## run the chain

context = "\n\n".join([doc[0].page_content for doc in chunks])


# Pass the user input
response = answer_chain.invoke({
        "context": context,
        "question": query
    })
print(response)

Global EV sales are performing strongly, surpassing 14 million units in 2023, which accounts for nearly 18% of all new car sales. The market is particularly led by Asia, especially China, while Europe and North America are also experiencing rapid growth.
